## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon, box, LineString, MultiPolygon
from shapely.ops import transform, unary_union
from shapely.validation import make_valid

import os
import requests
import tempfile
from pyproj import CRS
import math
import h3

import pyproj
from functools import partial

### Storm polygon generation

In [ ]:
def create_hurricane_polygon_shapefiles(csv_path, output_dir):
    """
    Creates polygon shapefiles for hurricane tracks using radius data.
    Processes ALL storms without filtering by region or removing any rows.
    """
    # Read the CSV file without dropping any rows
    df = pd.read_csv(csv_path)
    print(f"Read CSV with {len(df)} rows, {len(df['SID'].unique())} unique storms")
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get unique storm IDs
    unique_sids = df['SID'].unique()
    processed_count = 0
    failed_count = 0
    
    for sid in unique_sids:
        # Filter data for this specific storm
        sid_df = df[df['SID'] == sid].copy().reset_index(drop=True)
        print(f"Processing SID: {sid} - Points: {len(sid_df)}")
        
        # Process all storms with at least one point
        if len(sid_df) >= 1:
            # Store individual point polygons
            point_polygons = []
            
            for i in range(len(sid_df)):
                try:
                    row = sid_df.iloc[i]
                    
                    # Get center point coordinates
                    center_lon = float(row['LON'])
                    center_lat = float(row['LAT'])
                    
                    # Default radius (in nautical miles) if we can't get valid values
                    ne_radius = se_radius = sw_radius = nw_radius = 10.0
                    
                    # Try to get radius values safely - if any fail, keep the default
                    try:
                        if 'USA_R50_NE' in row and pd.notna(row['USA_R50_NE']):
                            val = str(row['USA_R50_NE']).strip()
                            if val != '':
                                ne_radius = float(val)
                    except: pass
                    
                    try:
                        if 'USA_R50_SE' in row and pd.notna(row['USA_R50_SE']):
                            val = str(row['USA_R50_SE']).strip()
                            if val != '':
                                se_radius = float(val)
                    except: pass
                    
                    try:
                        if 'USA_R50_SW' in row and pd.notna(row['USA_R50_SW']):
                            val = str(row['USA_R50_SW']).strip()
                            if val != '':
                                sw_radius = float(val)
                    except: pass
                    
                    try:
                        if 'USA_R50_NW' in row and pd.notna(row['USA_R50_NW']):
                            val = str(row['USA_R50_NW']).strip()
                            if val != '':
                                nw_radius = float(val)
                    except: pass
                    
                    # Convert radius from nautical miles to degrees
                    lat_scale = 1/60.0  # 1 degree = 60 nautical miles
                    lat_radians = math.radians(center_lat)
                    
                    # Avoid division by zero for poles
                    cos_lat = math.cos(lat_radians)
                    if abs(cos_lat) < 0.001:
                        cos_lat = 0.001 if cos_lat >= 0 else -0.001
                    
                    lon_scale = 1/(60.0 * cos_lat)
                    
                    # Create polygon points
                    polygon_points = []
                    num_segments = 36  # For a smooth circle-like shape
                    
                    for j in range(num_segments):
                        angle = j * (2 * math.pi / num_segments)
                        
                        # Determine radius based on the angle
                        if 0 <= angle < math.pi/2:  # NE
                            radius = ne_radius
                        elif math.pi/2 <= angle < math.pi:  # SE
                            radius = se_radius
                        elif math.pi <= angle < 3*math.pi/2:  # SW
                            radius = sw_radius
                        else:  # NW
                            radius = nw_radius
                        
                        # Calculate point coordinates
                        point_lon = center_lon + radius * lon_scale * math.cos(angle)
                        point_lat = center_lat + radius * lat_scale * math.sin(angle)
                        polygon_points.append((point_lon, point_lat))
                    
                    # Close the polygon
                    polygon_points.append(polygon_points[0])
                    
                    # Create polygon for this point
                    point_polygon = Polygon(polygon_points)
                    if point_polygon.is_valid:
                        point_polygons.append(point_polygon)
                    
                except Exception as e:
                    print(f"  Error processing point {i} for SID {sid}: {str(e)}")
            
            # Create the final polygon if we have any valid points
            if point_polygons:
                try:
                    # Choose polygon creation method
                    if len(point_polygons) == 1:
                        # Just use the single polygon
                        storm_polygon = point_polygons[0]
                        method_used = "Single Point"
                    else:
                        # Try to create a union of all polygons
                        try:
                            storm_polygon = unary_union(point_polygons)
                            method_used = "Point Union"
                            
                            # If we got a MultiPolygon, take the largest part
                            if isinstance(storm_polygon, MultiPolygon):
                                largest_polygon = max(storm_polygon.geoms, key=lambda p: p.area)
                                storm_polygon = largest_polygon
                                method_used = "Largest Polygon"
                                
                        except Exception as e:
                            # Fallback to convex hull if union fails
                            print(f"  Union failed for SID {sid}: {str(e)}")
                            all_points = []
                            for poly in point_polygons:
                                all_points.extend(list(poly.exterior.coords))
                            storm_polygon = Polygon(all_points).convex_hull
                            method_used = "Convex Hull"
                    
                    # Create metadata from first point
                    metadata = {}
                    for col in sid_df.columns:
                        if col not in ['LON', 'LAT', 'geometry']:
                            val = sid_df.iloc[0].get(col, '')
                            if pd.notna(val):
                                metadata[col] = val
                    
                    # Add some useful metadata
                    metadata['SID'] = sid
                    metadata['method'] = method_used
                    
                    # Add some calculated metrics if available
                    try:
                        if 'USA_WIND' in sid_df.columns:
                            winds = [float(w) for w in sid_df['USA_WIND'] if pd.notna(w)]
                            if winds:
                                metadata['max_wind'] = max(winds)
                    except: pass
                    
                    try:
                        if 'USA_PRES' in sid_df.columns:
                            pres = [float(p) for p in sid_df['USA_PRES'] if pd.notna(p)]
                            if pres:
                                metadata['min_pres'] = min(pres)
                    except: pass
                    
                    try:
                        if 'USA_SSHS' in sid_df.columns:
                            sshs = [float(s) for s in sid_df['USA_SSHS'] if pd.notna(s) and float(s) >= 0]
                            if sshs:
                                metadata['max_sshs'] = max(sshs)
                    except: pass
                    
                    # Clean up metadata keys for shapefile compatibility
                    clean_metadata = {}
                    for key, value in metadata.items():
                        # Limit to 10 characters, alphanumeric + underscore
                        new_key = ''.join(c for c in key if c.isalnum() or c == '_')[:10]
                        clean_metadata[new_key] = value
                    
                    # Create GeoDataFrame
                    polygon_gdf = gpd.GeoDataFrame(
                        [clean_metadata],
                        geometry=[storm_polygon],
                        crs="EPSG:4326"
                    )
                    
                    # Write to shapefile
                    output_file = os.path.join(output_dir, f'{sid}_hurricane_polygon.shp')
                    polygon_gdf.to_file(output_file)
                    print(f"  Connected polygon shapefile created for SID {sid}: {output_file}")
                    processed_count += 1
                    
                except Exception as e:
                    print(f"  Failed to create shapefile for SID {sid}: {str(e)}")
                    failed_count += 1
            else:
                print(f"  No valid polygons created for SID {sid}")
                failed_count += 1
        else:
            print(f"  No points found for SID {sid}")
            failed_count += 1
    
    print(f"\nProcessed {processed_count} storms successfully, {failed_count} failed")
    print("All connected polygon shapefiles have been generated.")

# Example usage
csv_path = 'global_storm_data.csv'
output_dir = 'hurricane_connected_polygon_shapefiles_fixed'
create_hurricane_polygon_shapefiles(csv_path, output_dir)

### Sort hurricane tracks to each basin/subbasin they pass through

In [ ]:
# Directories containing the shapefiles
hurricane_dir = 'hurricane_connected_polygon_shapefiles_fixed' ## 'hurricane_line_shapefiles2' ## hurricane_line_shapefiles3
polygon_dir = 'polygon_shapefiles' ## 'polygon_shapefiles'

# List of polygon identifiers
polygon_names = ['MOZ_MAD', 'BAY_BEN', 'EST_AUS', 'WST_AUS', 
                 'CAR_SEA', 'GUF_MEX', 'STN_ISL', 'PHI_PIN', 'EQU_CON']

# Create directories for each polygon
output_dirs = {name: os.path.join('categorized_polygon_shapefiles2', name) for name in polygon_names}  ## 'categorized_line_shapefiles2'
for directory in output_dirs.values():
    os.makedirs(directory, exist_ok=True)

# Load all polygon shapefiles into a single GeoDataFrame with an identifier column
polygon_files = [f for f in os.listdir(polygon_dir) if f.endswith('.shp')]
polygons_list = []

for file in polygon_files:
    name = os.path.splitext(file)[0]
    if name in polygon_names:
        gdf = gpd.read_file(os.path.join(polygon_dir, file))
        gdf['polygon_name'] = name  # Add an identifier column
        polygons_list.append(gdf)

polygons_gdf = gpd.GeoDataFrame(pd.concat(polygons_list, ignore_index=True))

# Ensure CRS is consistent
polygons_gdf = polygons_gdf.to_crs("EPSG:4326")

# Process each hurricane shapefile
hurricane_files = [f for f in os.listdir(hurricane_dir) if f.endswith('.shp')]

for hurricane_file in hurricane_files:
    # Load the hurricane shapefile
    hurricane_gdf = gpd.read_file(os.path.join(hurricane_dir, hurricane_file))
    
    # Ensure CRS is consistent
    hurricane_gdf = hurricane_gdf.to_crs("EPSG:4326")

    # Check intersections with polygons
    intersections = polygons_gdf[polygons_gdf.geometry.intersects(hurricane_gdf.geometry.union_all())]
    
    if not intersections.empty:
        # Save to the folder corresponding to the first intersecting polygon
        for _, row in intersections.iterrows():
            polygon_name = row['polygon_name']
            output_path = os.path.join(output_dirs[polygon_name], hurricane_file)
            hurricane_gdf.to_file(output_path)
            print(f"{hurricane_file} saved to {polygon_name} folder.")
    else:
        print(f"{hurricane_file} does not intersect with any polygon.")

print("Categorization complete.")


### Combine files

In [ ]:
def combine_shapefiles_in_categories(categorized_dir):
    # Get all subdirectories in the categorized directory
    categories = [d for d in os.listdir(categorized_dir) 
                  if os.path.isdir(os.path.join(categorized_dir, d))]
    
    for category in categories:
        category_path = os.path.join(categorized_dir, category)
        
        # Find all shapefiles in the category
        shapefiles = [f for f in os.listdir(category_path) if f.endswith('.shp')]
        
        # Collect GeoDataFrames
        gdfs = []
        for shapefile in shapefiles:
            gdf = gpd.read_file(os.path.join(category_path, shapefile))
            gdfs.append(gdf)
        
        # Combine shapefiles
        if gdfs:
            combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
            combined_gdf.crs = "EPSG:4326"
            
            # Save combined shapefile
            output_path = os.path.join(categorized_dir, f'{category}_combined_poly.shp')
            combined_gdf.to_file(output_path)
            print(f"Combined shapefile created for {category}: {output_path}")

# Example usage
categorized_dir = 'categorized_polygon_shapefiles2'
combine_shapefiles_in_categories(categorized_dir)

# Hexgrids

In [ ]:
def create_hexagon(center_x, center_y, size):
    """Create a hexagon of given size centered at (center_x, center_y)"""
    # Start from top point and work clockwise
    angles_deg = np.array([30, 90, 150, 210, 270, 330])
    angles_rad = np.radians(angles_deg)
    x = center_x + size * np.cos(angles_rad)
    y = center_y + size * np.sin(angles_rad)
    return Polygon(zip(x, y))

def generate_hex_grid(bounds, target_area_km2):
    """Generate a continuous hexagonal grid within given bounds"""
    minx, miny, maxx, maxy = bounds
    
    # Calculate center point for projections
    center_lat = (miny + maxy) / 2
    center_lon = (minx + maxx) / 2
    
    # Create an equal area projection centered on the region
    proj_string = f"+proj=aea +lat_1={center_lat-10} +lat_2={center_lat+10} +lat_0={center_lat} +lon_0={center_lon} +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Create transformation functions
    project = pyproj.Transformer.from_crs('EPSG:4326', proj_string, always_xy=True).transform
    unproject = pyproj.Transformer.from_crs(proj_string, 'EPSG:4326', always_xy=True).transform
    
    # Transform bounds to projected space
    bbox = transform(project, box(minx, miny, maxx, maxy))
    proj_bounds = bbox.bounds
    
    # Calculate hexagon size
    hex_size = np.sqrt(2 * target_area_km2 * 1e6 / (3 * np.sqrt(3)))  # Convert km² to m²
    
    # Calculate spacing
    width = proj_bounds[2] - proj_bounds[0]
    height = proj_bounds[3] - proj_bounds[1]
    
    # Horizontal spacing between hexagon centers
    dx = hex_size * np.sqrt(3)
    # Vertical spacing between hexagon centers
    dy = hex_size * 1.5
    
    # Calculate number of columns and rows needed
    cols = int(np.ceil(width / dx)) + 2  # Add buffer
    rows = int(np.ceil(height / dy)) + 2  # Add buffer
    
    # Adjust bounds to ensure coverage
    x_start = proj_bounds[0] - hex_size
    y_start = proj_bounds[1] - hex_size
    
    hexagons = []
    for row in range(rows):
        # Offset every other row
        if row % 2 == 0:
            x_offset = 0
        else:
            x_offset = dx / 2
            
        for col in range(cols):
            center_x = x_start + (col * dx) + x_offset
            center_y = y_start + (row * dy)
            
            # Create hexagon in projected space
            hex_poly = create_hexagon(center_x, center_y, hex_size)
            # Transform back to geographic coordinates
            geo_hex = transform(unproject, hex_poly)
            hexagons.append(geo_hex)
    
    return gpd.GeoDataFrame({'geometry': hexagons}, crs='EPSG:4326')

def print_region_stats(hex_grid):
    """Print detailed statistics for each region"""
    print("\nDetailed Region Statistics:")
    print("-" * 50)
    for region in sorted(hex_grid['region'].unique()):
        region_data = hex_grid[hex_grid['region'] == region]
        print(f"\nRegion: {region}")
        print(f"Number of hexagons: {len(region_data)}")
        print(f"Average area: {region_data['area_km2'].mean():.2f} km²")
        print(f"Min area: {region_data['area_km2'].min():.2f} km²")
        print(f"Max area: {region_data['area_km2'].max():.2f} km²")
        print(f"Standard deviation: {region_data['area_km2'].std():.2f} km²")

def create_multi_region_hex_grid(regions, target_area_km2=100):
    all_hexagons = []
    grid_id = 1
    
    for region_name, coordinates in regions.items():
        print(f"Processing region: {region_name}")
        # Create region polygon
        region_poly = Polygon(coordinates)
        
        # Generate hex grid for region bounds
        hex_grid = generate_hex_grid(region_poly.bounds, target_area_km2)
        
        # Create region-specific equal area projection
        center_lat = region_poly.centroid.y
        center_lon = region_poly.centroid.x
        proj_string = f"+proj=aea +lat_1={center_lat-15} +lat_2={center_lat+15} +lat_0={center_lat} +lon_0={center_lon} +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
        
        # Project region and hexagons for accurate intersection test and area calculation
        project = pyproj.Transformer.from_crs('EPSG:4326', proj_string, always_xy=True).transform
        proj_region = transform(project, region_poly)
        
        # Filter hexagons that intersect with region
        region_hexagons = []
        for hex_poly in hex_grid.geometry:
            # Project hexagon to same CRS as region
            proj_hex = transform(project, hex_poly)
            
            if proj_hex.intersects(proj_region):
                centroid = hex_poly.centroid
                # Calculate area in the projected CRS
                area_km2 = proj_hex.area / 1e6  # Convert m² to km²
                region_hexagons.append({
                    'geometry': hex_poly,
                    'grid_id': grid_id,
                    'region': region_name,
                    'centroid_lon': centroid.x,
                    'centroid_lat': centroid.y,
                    'area_km2': area_km2
                })
                grid_id += 1
        
        all_hexagons.extend(region_hexagons)
        print(f"Added {len(region_hexagons)} hexagons for {region_name}")
    
    # Create GeoDataFrame with all hexagons
    hex_grid = gpd.GeoDataFrame(all_hexagons, crs='EPSG:4326')
    
    return hex_grid

# Define regions (your existing regions dictionary here)

# Create grid for all regions
hex_grid = create_multi_region_hex_grid(regions, target_area_km2=100)

# Print detailed statistics
print_region_stats(hex_grid)

# Save separate files for each region
for region_name in regions.keys():
    region_grid = hex_grid[hex_grid['region'] == region_name]
    region_grid.to_file(f"{region_name}_hexgrid.shp")
    print(f"Saved {region_name} with {len(region_grid)} hexagons")

print(f"\nOverall Results:")
print(f"Average hexagon area: {hex_grid['area_km2'].mean():.2f} km²")